In [2]:
import ast
import base64
import csv
import enum
import glob
import json
import logging
import os
import os.path as osp
import pickle
import sys
from collections import defaultdict

import colorama
import datasets
import numpy as np
import pandas as pd
import toml
import torch
import transformers
from datasets import Dataset, load_dataset
from icecream import ic
from tqdm import *
from tqdm.auto import tqdm
import argparse

In [19]:
feature_path = "/home/yangliu/data/vinvl/model_0060000/coco2014test"
tsv_path = osp.expanduser(f"{feature_path}/features.tsv")
pred_path = osp.expanduser(f"{feature_path}/predictions.tsv")
lidx_path = osp.expanduser(f"{feature_path}/features.lineidx")
imgid2idx_path = osp.expanduser(f"{feature_path}/imageid2idx.json")

In [29]:
def transform_dict(d: dict) -> dict:
    keys = d.items()[0].keys()
    ic(keys)
    d_new = defaultdict(list)
    for k, vdict in d.items(): 
        for k_, v in vdict:
            d_new[k_].append(v)
    return d_new

def _getid(image_path):
    r"""
    "VizWiz-werwerwer-0001123.jpg" => 1123
    "COCO_train2014_00000000123123.jpg" => 123123
    """
    filename = os.path.basename(image_path)
    image_id = filename.split("_")[-1]
    image_id = image_id.split(".")[0]
    return int(image_id)

In [21]:
with open(imgid2idx_path, "rt") as f:
    imgid2idx = json.load(f)
    total_lines = max([idx for imgid, idx in imgid2idx.items()]) + 1
    ic(total_lines)

ic| total_lines: 40775


In [30]:
record_features = dict()
with open(tsv_path) as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
    for line in tqdm(tsvreader, total=total_lines):
        image_id, num_boxes, feature_enc = line[0], int(line[1]), line[-1]
        try:
            image_id = int(image_id)
        except ValueError:
            # not int but image name
            image_id = _getid(image_id)
        decoded = np.frombuffer(base64.b64decode(feature_enc), np.float32).reshape(
            num_boxes, -1
        )
        ic(image_id, num_boxes, decoded.shape)
        record_features[image_id] = decoded
        break

  0%|          | 0/40775 [00:00<?, ?it/s]

ic| image_id: 1, num_boxes: 38, decoded.shape: (38, 2054)
